___
# Phase 2 - Milestone 2
    Pratama Waskito Prihandana
    Full Time Data Science
    

____
# **I. Library**

In [1]:
!pip install num2words
!pip install pyspellchecker
!pip install spacy
!pip install xgboost
!pip install gensim
!pip install translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=48b2d299f1dd7760c0bd1a0e045ec1acbe345b3e5eb8c377b8d196ef0fbe5937
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexe

# **II. Load Model**

In [3]:
from google.colab import files
ensemble_classifier_load = files.upload()

Saving ensemble_classifier.pkl to ensemble_classifier.pkl


In [4]:
from google.colab import files
naivebayes_load = files.upload()

Saving multinomial_naivebayes.pkl to multinomial_naivebayes.pkl


In [5]:
from google.colab import files
model1_rnn_load = files.upload()

Saving model1_rnn_sequential.h5 to model1_rnn_sequential.h5


In [6]:
from google.colab import files
model2_rnn_load = files.upload()

Saving model2_rnn_sequential.h5 to model2_rnn_sequential.h5


In [10]:
from google.colab import files
data_inference = files.upload()

Saving SMS_test.csv to SMS_test.csv


In [30]:
from google.colab import files
text_preprocess = files.upload()

Saving preprocessor.pkl to preprocessor (1).pkl


In [34]:
import pickle
import keras

# ensemble model
with open('ensemble_classifier.pkl', 'rb') as file_1:
  ensemble_classifier = pickle.load(file_1)

# multinomial naive bayes model
with open('multinomial_naivebayes.pkl', 'rb') as file_2:
  nb = pickle.load(file_2)

# model1 RNN
model1_rnn = keras.models.load_model('model1_rnn_sequential.h5')

# model2 RNN
model2_rnn = keras.models.load_model('model2_rnn_sequential.h5')

____
# **III. Model Implementation**

In [19]:
import pandas as pd

df = pd.read_csv('SMS_test.csv',  encoding = 'latin_1')

In [20]:
df.head()

,S. No.,Message_body,Label
0,1,"UpgrdCentre Orange customer, you may now claim...",Spam
1,2,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,3,Congrats! Nokia 3650 video camera phone is you...,Spam
3,4,URGENT! Your Mobile number has been awarded wi...,Spam
4,5,Someone has contacted our dating service and e...,Spam


In [24]:
# manual encoding on target
label_category = {'Non-Spam': 0, 'Spam': 1}
df.replace({'label': label_category}, inplace = True)

# remove unnecessary column
df = df.drop('S. No.', axis=1)

In [25]:
df.head()

,Message_body,Label
0,"UpgrdCentre Orange customer, you may now claim...",Spam
1,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,Congrats! Nokia 3650 video camera phone is you...,Spam
3,URGENT! Your Mobile number has been awarded wi...,Spam
4,Someone has contacted our dating service and e...,Spam


In [46]:
import string
import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
# Preprocessing the text data

def preprocess_text(text):
    # Ensure that the input is a string
    if not isinstance(text, str):
        raise ValueError("Input must be a string")
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert text to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub('\d+', '', text)
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    return text

# Tokenizing and stemming the text data
def tokenize_and_stem(text):
    stemmer = SnowballStemmer("english")
    tokens = text.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

# Lemmatizing the text data
nltk.download('wordnet')
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# Removing stop words from the text data
nltk.download('stopwords')
def remove_stopwords(text):
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))
    return " ".join([word for word in str(text).split() if word not in stopwords])

preprocessed_text = df['Message_body'].apply(preprocess_text)
tokenized_text = preprocessed_text.apply(tokenize_and_stem)
lemmatized_text = tokenized_text.apply(lemmatize)
complete_preprocess = lemmatized_text.apply(remove_stopwords)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
df['mes_norm'] = complete_preprocess

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
xx = vectorizer.fit_transform(complete_preprocess)

In [83]:
nb.fit(xx, df['Label'])
nb_pred = nb.predict(xx)

In [106]:
df['NaiveBayes_pred'] = nb_pred
df2 = pd.concat([df[['Message_body']], df[['Label']], df[['NaiveBayes_pred']]], axis=1)

In [108]:
df2.head(50)

,Message_body,Label,NaiveBayes_pred
0,"UpgrdCentre Orange customer, you may now claim...",Spam,Spam
1,"Loan for any purpose £500 - £75,000. Homeowner...",Spam,Spam
2,Congrats! Nokia 3650 video camera phone is you...,Spam,Spam
3,URGENT! Your Mobile number has been awarded wi...,Spam,Spam
4,Someone has contacted our dating service and e...,Spam,Spam
5,Send a logo 2 ur lover - 2 names joined by a h...,Spam,Spam
6,FREE entry into our £250 weekly competition ju...,Spam,Spam
7,100 dating service cal;l 09064012103 box334sk38ch,Spam,Spam
8,FREE RINGTONE text FIRST to 87131 for a poly o...,Spam,Spam
9,4mths half price Orange line rental & latest c...,Spam,Spam
